In [ ]:
import os
import json

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import rc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
cwd = os.getcwd()
REPORT_PATH = os.path.join(cwd, "..", "..", "reports", "03_D2.4_appendix2")
DATA_PATH = os.path.join(cwd, "..", "..", "data", "input", "processed")
VAR_DF_PATH = os.path.join(cwd, "..", "..", "data", "input", "raw", "vars_list_details_and_tags.xlsx")
SHP_PATH = os.path.join(cwd, "..", "..", "data", "input", "processed", "shapefiles")

In [ ]:
regions_df = gpd.read_file(os.path.join(SHP_PATH, "LAU.shp"))
regions_df.drop(columns= ["year"], inplace=True)

In [ ]:
def processes_data(data_df):

    # merge dfs
    final_df = pd.merge(
        regions_df,
        data_df,
        left_on="prnt_code",
        right_on="reg_code",
        how="right",
    )

    final_df.drop(
        columns=[
            "reg_code",
            "name"
        ],
        inplace=True,
    )

    return final_df

In [ ]:
clm_impact_vars = ["probability of a heatwave",
                   "change in heatwave frequency",
                   "probability of a coldwave",
                   "change in coldwave frequency",
                   "probability of a drought",
                   "change in drought frequency"
                   ]

In [ ]:
clt_exps = ["Historical", "RCP2.6", "RCP6.0", "RCP8.5"]

In [ ]:
nl = '\n'
latex_nl = '\\\\'
esc_nl = '\\'

In [ ]:
replacements = {'begin{tabular}': 'begin{tabularx}{\\textwidth}',
                'end{tabular}': 'end{tabularx}',
                 "& 0 \\\\": "",
                "_": "\_",
                '\\\\': '\\\\\midrule',
                "{\\textwidth}{| X | X |}\n": "{\\textwidth}{| X | X |}\n\midrule"

               }

def get_latex_table(df):
    # prepare column_format
    col_format = "| X |"
    for i in range(len(df.columns)):
        col_format = f"{col_format} X |"
    
    # pd to latex
    table = df.style.to_latex(column_format=col_format, 
                                             position_float="centering",
                                             hrules=False,
                                             environment = "table*", 
                                             position="h"
                                             )
    # replace tabular with tabularx and add width=textwidth
    for key, value in replacements.items():
        table = table.replace(key, value)
        
    return table 

In [ ]:
col_dict = {0: "mistyrose", 1: "lightcoral", 2: "indianred", 3: "brown"}
map_dict = {0: "low", 1: "moderate", 2: "high", 3: "very high"}

In [ ]:
def stacked_bar_plot(x_list, y_nested_list, ax, x_tick_labels, legend_labels, width=0.2):

    y_pos = x_list[:]
    
    p1 = ax.bar(y_pos, y_nested_list[0], width, linewidth=0, color=col_dict[0])

    y_bottom = y_nested_list[0]

    p_list = [p1]
    
    for n in np.arange(1, len(x_list)):
        pn = ax.bar(y_pos, y_nested_list[n], width, bottom=y_bottom, linewidth=0, color=col_dict[n])

        p_list.append(pn)

        y_bottom = [y_bottom[i] + y_nested_list[n][i] for i in np.arange(len(x_list))]
    
    ax.set_xticks(x_list)
    ax.set_xticklabels(x_tick_labels)
    
    ax.set_axisbelow(True)
    ax.yaxis.grid(color='gray', linestyle='-', alpha=0.3)
    
    pn0_set_for_legend = (p[0] for p in p_list)

    ax1.legend(pn0_set_for_legend, legend_labels, bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left",
       mode="expand", borderaxespad=0, ncol=4)

In [ ]:
#table and BOX plots
for var_name in clm_impact_vars:
    var_unit = "number"

    table_dict = {"Unit of measure": var_unit, 
                  "Original resolution": "NUTS3",
                  "Further disaggregated?": "yes",
                  "Disaggregation method": "Same value as parent NUTS3 region to its LAU regions",
                  "Data source": 'Projecting Exposure to Extreme Climate Impact Events Across Six Event Categories and Three Spatial Scales'
                          }
    
    table = pd.DataFrame.from_dict(table_dict, orient="index")
    with open(os.path.join(REPORT_PATH,  "tables", f"{var_name[0].upper()}{var_name[1:]}.tex"), "w") as f:

        latex_table = get_latex_table(table)
        f.write(latex_table)
        f.write(nl)
    
    data_df = pd.read_csv(os.path.join(DATA_PATH, var_name, "data.csv"))
    data_df = processes_data(data_df)
    
    for country_code in ["DE", "PL", "ES"]:
        sub_df = data_df[data_df["prnt_code"].str.startswith(country_code)]

        fig = plt.figure(figsize=(5, 4))
        gs = fig.add_gridspec(1, 1, wspace=0, hspace=0)

        ax1 = plt.subplot(gs[:, :]) #-------------------------------
        
        if "probability" in var_name:
            sns.boxplot(x = sub_df["climate_experiment"] ,
                        y = sub_df["value"],
                        width=0.3, ax=ax1)

            ax1.set_ylabel(f"[{var_unit}]", fontsize=13)
            ax1.set_xlabel("Climate scenario", fontsize=13)
        
        else: 
            y_list = []
            for intensity in [0, 1, 2, 3]:
                group = sub_df[sub_df["value"] == intensity]
                y_list.append(
                    [len(group[group["climate_experiment"] == "Historical"]),
                     len(group[group["climate_experiment"] == "RCP2.6"]),
                     len(group[group["climate_experiment"] == "RCP6.0"]),
                     len(group[group["climate_experiment"] == "RCP8.5"]),
                     ])
                
            y_max = len(sub_df[sub_df["climate_experiment"] == "Historical"]) + 500
            ax1.set_ylim([0, y_max])
            
            stacked_bar_plot([0, 1, 2, 3], y_list, ax1, clt_exps, legend_labels=["low", 'moderate', 'high', 'very high'])
            ax1.set_ylabel(f"[number of regions]", fontsize=15)
            ax1.set_xlabel("Climate scenario", fontsize=15)
            
        ax1.tick_params(labelsize=11)
        plt.savefig(os.path.join(REPORT_PATH, 
                                 "figures", 
                                 f"{var_name[0].upper()}{var_name[1:]}_{country_code}_box.png"),
                   format='png', bbox_inches="tight", dpi=200)
    

In [ ]:
# maps
for var_name in clm_impact_vars:

    data_df = pd.read_csv(os.path.join(DATA_PATH, var_name, "data.csv"))
    data_df = processes_data(data_df)
    var_unit = "number"
        
    for country_code in ["DE", "PL", "ES"]:
        sub_df = data_df[data_df["prnt_code"].str.startswith(country_code)]
        
        fig = plt.figure(figsize=(10, 10))
        gs = fig.add_gridspec(2, 2, wspace=0, hspace=0.3)
        
        
        # set min and max values across all subplots
        val_list = list(sub_df['value'])
        vmin, vmax = min(val_list), max(val_list) 

        clt_ax_dict = {"Historical": plt.subplot(gs[:1, :1]), 
                       "RCP2.6": plt.subplot(gs[:1, 1:]),  
                       "RCP6.0": plt.subplot(gs[1:, :1]),
                       "RCP8.5": plt.subplot(gs[1:, 1:]) }
        
        for clt_exp, ax in clt_ax_dict.items():

            _sub_df = sub_df[sub_df["climate_experiment"] == clt_exp]

            if "probability" in var_name:
                _sub_df.plot(column='value', 
                             cmap='Blues', 
                             linewidth=0, 
                             ax=ax, 
                             edgecolor='none', 
                             norm=plt.Normalize(vmin=vmin, vmax=vmax))

            else:
                for key, group in _sub_df.groupby("value"): 
                    group.plot(column='value', 
                               color=col_dict[key], 
                               edgecolor=col_dict[key],
                               ax=ax)
        
            ax.set_title(f"Climate scenario:\n{clt_exp}")
            ax.tick_params(
                axis='x',          
                which='both',      
                bottom=False,      
                top=False,        
                labelbottom=False) 

            ax.tick_params(
                axis='y',          
                which='both',      
                left=False,     
                right=False,         
                labelleft=False) 

            ax.set_frame_on(False)

        if "probability" in var_name:
        
            sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
            sm._A = []
            axins = inset_axes(plt.subplot(gs[1:, 1:]),
                        width="100%",  
                        height="4%",
                        loc='lower center',
                        borderpad=-1
                       )
            clb=fig.colorbar(sm, 
                         cax=axins, 
                         orientation="horizontal",
                        )
            clb.ax.set_title(f"[{var_unit}]") #, rotation = 90, x=2.5, y=1
        else:
            handles = []
            for col, color in col_dict.items():
                patch = mpatches.Patch(color=color, label=map_dict[col])
                handles.append(patch)
            plt.legend(handles=handles, bbox_to_anchor=(-1, -0.2, 1, 3), loc="lower left",
                       mode="expand", borderaxespad=1, ncol=2)
                               
                
        plt.savefig(os.path.join(REPORT_PATH, 
                                 "figures", 
                                 f"{var_name[0].upper()}{var_name[1:]}_{country_code}_map.png"),
                   format='png', bbox_inches="tight", dpi=200)